<a href="https://colab.research.google.com/github/hammy229/SMU-Low-SES-Extraction-Work/blob/main/Doshi_Low_SES_LR_LSTM_CNN_SVM_Naive_Bayes_KNN_Decision_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Move to your shared file**

In [ ]:
%cd /content/drive/My Drive/SMU Work

/content/drive/My Drive/SMU Work


In [ ]:
import os
path = "GroundTruthFilter"
files = [file for file in os.listdir(path) if not file.startswith('.')]

In [ ]:
path = "GroundTruthFilter"  # assume this is the folder containing your ground truth files
files = [file for file in os.listdir(path) if not file.startswith('.')]

ground_truth_texts = []
for file_name in files:
    with open(os.path.join(path, file_name), 'r') as f:
        output = f.read()
        ground_truth_texts.append(output)

In [ ]:
ground_truth_texts = []

In [ ]:
len(ground_truth_texts)

0

In [ ]:
for file_name in files:
    with open(os.path.join(path, file_name), 'r') as f:  # Use os.path.join to construct the correct file path
        output = f.read()
        ground_truth_texts.append(output)

In [ ]:
import nltk
# Preprocessing text data
nltk.download('stopwords')
nltk.download('punkt')  # Ensure 'punkt' is downloaded
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    tokens = word_tokenize(text.lower())  # Tokenization and lowercasing
    tokens = [word for word in tokens if word.isalpha()]  # Remove non-alphabetic tokens
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    return ' '.join(tokens)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
ground_truth_texts= [preprocess_text(item) for item in ground_truth_texts]

In [ ]:
my_file = open("not-low-ses.txt", "r")
data = my_file.readlines()
my_file.close()

In [ ]:
len(data)

468

In [ ]:
not_low_ses = []
for line in data:
    if len(line)>100:
        not_low_ses.append(line)
    if len(not_low_ses) == len(ground_truth_texts):
        break

In [ ]:
len(not_low_ses)

64

In [ ]:
not_low_ses_texts= [preprocess_text(item) for item in not_low_ses]

In [ ]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
all_text = ground_truth_texts + not_low_ses_texts

In [ ]:
len(all_text)

128

In [ ]:
all_label = [1 for i in range(len(ground_truth_texts))] + [-1 for i in range(len(not_low_ses_texts))]

In [ ]:
len(all_label)

128

In [ ]:
# all_text

# **Simple Classifier**

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

all_text = np.array(all_text)
all_label = np.array(all_label)

train_texts, val_texts, train_labels, val_labels = train_test_split(all_text, all_label, test_size=0.2, random_state=42, shuffle=True)

In [ ]:
vectorizer.fit(train_texts)
X_train = vectorizer.transform(train_texts)
X_test  = vectorizer.transform(val_texts)

In [ ]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(X_train, train_labels)
score = classifier.score(X_test, val_labels)

print("Accuracy:", score)


Accuracy: 0.8076923076923077


In [ ]:
classifier.predict_proba(X_train[0:3])

array([[0.975454  , 0.024546  ],
       [0.98770356, 0.01229644],
       [0.0072068 , 0.9927932 ]])

In [ ]:
text = "A lot of people around me at this school come from families that live comfortably. I, however, am poor. Like my mom makes less than 30k a year. I feel like people are hearing me but they’re not listening. We’re also at a disadvantage because the schooling in my area isn’t as rigorous as LA or Bay Area kids. Nonetheless, I’m here. My tuition or room and board is expensive. Not even 33k worth of grants and scholarships and loans could pay for all of my school. And I also didn’t come to college with a laptop. I don’t have enough money for books. How the hell am I supposed to be an engineer without a laptop. That’s like the 1 thing I need. This sucks hard. It’s a fun adventure, but the real world is fucking brutal."

In [ ]:
def LRPredict(text):
    text = preprocess_text(text)
    vec_text = vectorizer.transform([text])
    prob = classifier.predict_proba(vec_text)
    if prob[0][1]>0.90:
        return "low_ses"
    else:
        return "not_low_ses"

In [ ]:
LRPredict(text)

'low_ses'

In [ ]:
def LRPredict(text):
    text = preprocess_text(text)
    vec_text = vectorizer.transform([text])
    prob = classifier.predict_proba(vec_text)[0][1]
    return prob

text = "My college had a student run food bank on campus (actually was pretty awesome, I volunteered there a bunch when I could.) We also got free bus/train passes, free gym membership, free food and tshirts at events at least once a week, career center workshops like resume building, free tutoring services, free editing for essays, even a student run “fix it Friday” where people would help fix broken phones and stuff. Healthcare center gave free condoms. Our library would give free scantrons and blue books (required for exams and can be like 10 bucks each) if you asked nicely."
ses_score = LRPredict(text)
print("SES Score:", ses_score)

if ses_score < 0.3:
    print("Very low SES")
elif ses_score < 0.6:
    print("Low SES")
elif ses_score < 0.8:
    print("Moderate SES")
else:
    print("High SES")

SES Score: 0.15847678742582413
Very low SES


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/SMU Work

import os
path = "GroundTruthFilter"
files = [file for file in os.listdir(path) if not file.startswith('.')]

ground_truth_texts = []

for file_name in files:
    with open(os.path.join(path, file_name), 'r') as f:
        output = f.read()
        ground_truth_texts.append(output)

import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    tokens = word_tokenize(text.lower())  # Tokenization and lowercasing
    tokens = [word for word in tokens if word.isalpha()]  # Remove non-alphabetic tokens
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    return ' '.join(tokens)

ground_truth_texts = [preprocess_text(item) for item in ground_truth_texts]

my_file = open("not-low-ses.txt", "r")
data = my_file.readlines()
my_file.close()

not_low_ses = []
for line in data:
    if len(line) > 100:
        not_low_ses.append(line)
    if len(not_low_ses) == len(ground_truth_texts):
        break

not_low_ses_texts = [preprocess_text(item) for item in not_low_ses]

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Combine the ground truth and not low SES texts into a single dataset
all_text = ground_truth_texts + not_low_ses_texts
all_label = [1 for i in range(len(ground_truth_texts))] + [-1 for i in range(len(not_low_ses_texts))]

# Split the dataset into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(all_text, all_label, test_size=0.2, random_state=42, shuffle=True)

# Vectorize the text data
vectorizer = CountVectorizer()
vectorizer.fit(train_texts)
X_train = vectorizer.transform(train_texts)
X_test = vectorizer.transform(val_texts)

# Initialize and fit the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, train_labels)

# Calculate the accuracy of the model
score = rf_model.score(X_test, val_labels)
print("Accuracy:", score)

def RFPredict(text):
    text = preprocess_text(text)
    vec_text = vectorizer.transform([text])
    prob = rf_model.predict(vec_text)
    if prob[0] == 1:
        return "low_ses"
    else:
        return "not_low_ses"

# Example text prediction
text = "it's a nice place. if you don't have friends it can be hard to find fun things to do on the weekends, so definitely make an effort early on to meet interesting people so that you aren't stuck alone in your dorm on the weekends."
print(RFPredict(text))

def classify_ses(text):
    low_ses_keywords = [
        "poor", "low income", "unemployed", "financial aid", "pell grant",
        "food pantry", "working multiple jobs", "low wage", "scholarships",
        "struggling to pay", "couldn't afford to", "no money for", "in debt", "relying on grants",
        "parents can't afford to", "financial hardship", "living in poverty"
    ]

    text_lower = text.lower()

    for keyword in low_ses_keywords:
        if keyword in text_lower:
            return "Low SES"

    return "Not Low SES"

def process_files(directory):
    results = {}
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            with open(os.path.join(directory, filename), 'r') as file:
                content = file.read()
                result = classify_ses(content)
                results[filename] = result
    return results

# Example of processing a directory of files
directory_path = "/content/drive/My Drive/SMU Work"
results = process_files(directory_path)
print(results)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/SMU Work


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Accuracy: 0.6538461538461539
not_low_ses
{'not-low-ses.txt': 'Low SES'}


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/SMU Work

import os
path = "GroundTruthFilter"
files = [file for file in os.listdir(path) if not file.startswith('.')]

ground_truth_texts = []

for file_name in files:
    with open(os.path.join(path, file_name), 'r') as f:
        output = f.read()
        ground_truth_texts.append(output)

import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    tokens = word_tokenize(text.lower())  # Tokenization and lowercasing
    tokens = [word for word in tokens if word.isalpha()]  # Remove non-alphabetic tokens
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    return ' '.join(tokens)

ground_truth_texts = [preprocess_text(item) for item in ground_truth_texts]

my_file = open("not-low-ses.txt", "r")
data = my_file.readlines()
my_file.close()

not_low_ses = []
for line in data:
    if len(line) > 100:
        not_low_ses.append(line)
    if len(not_low_ses) == len(ground_truth_texts):
        break

not_low_ses_texts = [preprocess_text(item) for item in not_low_ses]

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

# Combine the ground truth and not low SES texts into a single dataset
all_text = ground_truth_texts + not_low_ses_texts
all_label = [1 for i in range(len(ground_truth_texts))] + [-1 for i in range(len(not_low_ses_texts))]

# Split the dataset into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(all_text, all_label, test_size=0.2, random_state=42, shuffle=True)

# Vectorize the text data
vectorizer = CountVectorizer()
vectorizer.fit(train_texts)
X_train = vectorizer.transform(train_texts)
X_test = vectorizer.transform(val_texts)

# Initialize and fit the Naive Bayes model
nb_model = MultinomialNB()
nb_model.fit(X_train, train_labels)

# Calculate the accuracy of the model
score = nb_model.score(X_test, val_labels)
print("Accuracy:", score)

def NBPredict(text):
    text = preprocess_text(text)
    vec_text = vectorizer.transform([text])
    prob = nb_model.predict(vec_text)
    if prob[0] == 1:
        return "low_ses"
    else:
        return "not_low_ses"

# Example text prediction
text = "it's a nice place. if you don't have friends it can be hard to find fun things to do on the weekends, so definitely make an effort early on to meet interesting people so that you aren't stuck alone in your dorm on the weekends."
print(NBPredict(text))

def classify_ses(text):
    low_ses_keywords = [
        "poor", "low income", "unemployed", "financial aid", "pell grant",
        "food pantry", "working multiple jobs", "low wage", "scholarships",
        "struggling to pay", "couldn't afford to", "no money for", "in debt", "relying on grants",
        "parents can't afford to", "financial hardship", "living in poverty"
    ]

    text_lower = text.lower()

    for keyword in low_ses_keywords:
        if keyword in text_lower:
            return "Low SES"

    return "Not Low SES"

def process_files(directory):
    results = {}
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            with open(os.path.join(directory, filename), 'r') as file:
                content = file.read()
                result = classify_ses(content)
                results[filename] = result
    return results

# Example of processing a directory of files
directory_path = "/content/drive/My Drive/SMU Work"
results = process_files(directory_path)
print(results)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/SMU Work


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Accuracy: 0.8076923076923077
not_low_ses
{'not-low-ses.txt': 'Low SES'}


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/SMU Work

import os
path = "GroundTruthFilter"
files = [file for file in os.listdir(path) if not file.startswith('.')]

ground_truth_texts = []

for file_name in files:
    with open(os.path.join(path, file_name), 'r') as f:
        output = f.read()
        ground_truth_texts.append(output)

import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    tokens = word_tokenize(text.lower())  # Tokenization and lowercasing
    tokens = [word for word in tokens if word.isalpha()]  # Remove non-alphabetic tokens
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    return ' '.join(tokens)

ground_truth_texts = [preprocess_text(item) for item in ground_truth_texts]

my_file = open("not-low-ses.txt", "r")
data = my_file.readlines()
my_file.close()

not_low_ses = []
for line in data:
    if len(line) > 100:
        not_low_ses.append(line)
    if len(not_low_ses) == len(ground_truth_texts):
        break

not_low_ses_texts = [preprocess_text(item) for item in not_low_ses]

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

# Combine the ground truth and not low SES texts into a single dataset
all_text = ground_truth_texts + not_low_ses_texts
all_label = [1 for i in range(len(ground_truth_texts))] + [-1 for i in range(len(not_low_ses_texts))]

# Split the dataset into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(all_text, all_label, test_size=0.2, random_state=42, shuffle=True)

# Vectorize the text data
vectorizer = CountVectorizer()
vectorizer.fit(train_texts)
X_train = vectorizer.transform(train_texts)
X_test = vectorizer.transform(val_texts)

# Initialize and fit the SVM model
svm_model = SVC(kernel='linear', C=1, random_state=42)
svm_model.fit(X_train, train_labels)

# Calculate the accuracy of the model
score = svm_model.score(X_test, val_labels)
print("Accuracy:", score)

def SVMPredict(text):
    text = preprocess_text(text)
    vec_text = vectorizer.transform([text])
    prob = svm_model.predict(vec_text)
    if prob[0] == 1:
        return "low_ses"
    else:
        return "not_low_ses"

# Example text prediction
text = "it's a nice place. if you don't have friends it can be hard to find fun things to do on the weekends, so definitely make an effort early on to meet interesting people so that you aren't stuck alone in your dorm on the weekends."
print(SVMPredict(text))

def classify_ses(text):
    low_ses_keywords = [
        "poor", "low income", "unemployed", "financial aid", "pell grant",
        "food pantry", "working multiple jobs", "low wage", "scholarships",
        "struggling to pay", "couldn't afford to", "no money for", "in debt", "relying on grants",
        "parents can't afford to", "financial hardship", "living in poverty"
    ]

    text_lower = text.lower()

    for keyword in low_ses_keywords:
        if keyword in text_lower:
            return "Low SES"

    return "Not Low SES"

def process_files(directory):
    results = {}
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            with open(os.path.join(directory, filename), 'r') as file:
                content = file.read()
                result = classify_ses(content)
                results[filename] = result
    return results

# Example of processing a directory of files
directory_path = "/content/drive/My Drive/SMU Work"
results = process_files(directory_path)
print(results)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/SMU Work


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Accuracy: 0.8076923076923077
not_low_ses
{'not-low-ses.txt': 'Low SES'}


In [ ]:
# Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

# Change directory to the SMU Work folder in Google Drive
%cd /content/drive/My Drive/SMU Work

# Import necessary libraries
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dropout
from tensorflow.keras.callbacks import Callback
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

# Download NLTK data (stopwords and tokenizer)
nltk.download('stopwords')
nltk.download('punkt')

# Preprocessing function to clean and tokenize text data
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    tokens = word_tokenize(text.lower())  # Convert text to lowercase and tokenize
    tokens = [word for word in tokens if word.isalpha()]  # Keep only alphabetic tokens (remove numbers, punctuation, etc.)
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords (common words like "and", "the", etc.)
    return ' '.join(tokens)  # Join tokens back into a single string

# Load Ground Truth Texts (low SES-related texts)
path = "GroundTruthFilter"
files = [file for file in os.listdir(path) if not file.startswith('.')]

ground_truth_texts = []
for file_name in files:
    with open(os.path.join(path, file_name), 'r') as f:
        output = f.read()
        ground_truth_texts.append(output)

# Preprocess ground truth texts
ground_truth_texts = [preprocess_text(item) for item in ground_truth_texts]

# Load texts that are not low SES-related
my_file = open("not-low-ses.txt", "r")
data = my_file.readlines()
my_file.close()

not_low_ses_texts = []
for line in data:
    if len(line) > 100:  # Filter lines with length greater than 100 characters
        not_low_ses_texts.append(line)
    if len(not_low_ses_texts) == len(ground_truth_texts):  # Ensure both categories have the same number of texts
        break

# Preprocess not low SES texts
not_low_ses_texts = [preprocess_text(item) for item in not_low_ses_texts]

# Combine both categories of texts and create labels
all_texts = ground_truth_texts + not_low_ses_texts
all_labels = [1] * len(ground_truth_texts) + [0] * len(not_low_ses_texts)  # 1 for low SES, 0 for not low SES

# Split the data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(all_texts, all_labels, test_size=0.2, random_state=42, shuffle=True)

# Tokenization and padding of sequences
vocab_size = 10000  # Maximum vocabulary size (number of unique words)
max_length = 200  # Maximum length of each sequence (number of words per text)

tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")  # Create tokenizer, using <OOV> for out-of-vocabulary words
tokenizer.fit_on_texts(train_texts)  # Fit the tokenizer on the training texts (learn the vocabulary)

# Convert the texts to sequences of integers (tokenized form)
train_sequences = tokenizer.texts_to_sequences(train_texts)
val_sequences = tokenizer.texts_to_sequences(val_texts)

# Pad the sequences to ensure uniform length (truncating or padding as necessary)
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating='post')
val_padded = pad_sequences(val_sequences, maxlen=max_length, padding='post', truncating='post')

# Define a custom callback to print only the epoch, accuracy, and dataset being trained on
class PrintEpochCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch {epoch + 1}: Training Accuracy: {logs['accuracy']:.4f}, Validation Accuracy: {logs['val_accuracy']:.4f}")

# LSTM Model for text classification
embedding_dim = 100  # Dimensionality of the embedding layer (size of the word vectors)
model_lstm = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length),  # Embedding layer for converting words to vectors
    LSTM(64, return_sequences=False),  # LSTM layer with 64 units (good for capturing sequential information in text)
    Dropout(0.5),  # Dropout layer to prevent overfitting
    Dense(32, activation='relu'),  # Dense (fully connected) layer with 32 units and ReLU activation
    Dropout(0.5),  # Another dropout layer for regularization
    Dense(1, activation='sigmoid')  # Output layer with a single unit and sigmoid activation for binary classification (1 or 0)
])

# Compile the LSTM model using binary cross-entropy loss and the Adam optimizer
model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the LSTM model on the training data, validating on the validation data, with custom callback
history_lstm = model_lstm.fit(train_padded, np.array(train_labels), epochs=10, validation_data=(val_padded, np.array(val_labels)), batch_size=32, callbacks=[PrintEpochCallback()])

# CNN Model for text classification
model_cnn = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length),  # Embedding layer
    Conv1D(128, 5, activation='relu'),  # 1D Convolutional layer with 128 filters and kernel size of 5
    MaxPooling1D(pool_size=2),  # Max pooling layer to down-sample the output of the convolutional layer
    Conv1D(128, 5, activation='relu'),  # Another 1D Convolutional layer
    GlobalMaxPooling1D(),  # Global max pooling layer to reduce the output to a single vector
    Dropout(0.5),  # Dropout layer for regularization
    Dense(32, activation='relu'),  # Dense layer with 32 units and ReLU activation
    Dropout(0.5),  # Another dropout layer for regularization
    Dense(1, activation='sigmoid')  # Output layer with a single unit and sigmoid activation for binary classification
])

# Compile the CNN model using binary cross-entropy loss and the Adam optimizer
model_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the CNN model on the training data, validating on the validation data, with custom callback
history_cnn = model_cnn.fit(train_padded, np.array(train_labels), epochs=10, validation_data=(val_padded, np.array(val_labels)), batch_size=32, callbacks=[PrintEpochCallback()])

# Evaluate the LSTM model on the validation data
loss_lstm, accuracy_lstm = model_lstm.evaluate(val_padded, np.array(val_labels))
print(f"LSTM Model Final Accuracy: {accuracy_lstm:.4f}")

# Evaluate the CNN model on the validation data
loss_cnn, accuracy_cnn = model_cnn.evaluate(val_padded, np.array(val_labels))
print(f"CNN Model Final Accuracy: {accuracy_cnn:.4f}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/SMU Work


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.4357 - loss: 0.6987Epoch 1: Training Accuracy: 0.4510, Validation Accuracy: 0.4231
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 134ms/step - accuracy: 0.4387 - loss: 0.6984 - val_accuracy: 0.4231 - val_loss: 0.6960
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.5375 - loss: 0.6891Epoch 2: Training Accuracy: 0.5196, Validation Accuracy: 0.4231
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.5339 - loss: 0.6894 - val_accuracy: 0.4231 - val_loss: 0.6980
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.5165 - loss: 0.6940 Epoch 3: Training Accuracy: 0.5294, Validation Accuracy: 0.4231
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.5191 - loss: 0.6942 - val_accuracy: 0.4231 - val_loss: 0.6985
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.4763 - loss: 0.6949Epoch 4: Training Accuracy: 0.5196, Validation Accuracy: 0.3846
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.4849 - loss: 0.6950 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Keep all the previous imports and data preprocessing steps

# Initialize and fit the KNN model
knn_model = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors
knn_model.fit(X_train, train_labels)

# Make predictions on the validation set
y_pred = knn_model.predict(X_test)

# Calculate the accuracy of the model
knn_accuracy = accuracy_score(val_labels, y_pred)
print("KNN Accuracy:", knn_accuracy)

def KNNPredict(text):
    text = preprocess_text(text)
    vec_text = vectorizer.transform([text])
    prob = knn_model.predict(vec_text)
    if prob[0] == 1:
        return "low_ses"
    else:
        return "not_low_ses"

# Example text prediction
text = "it's a nice place. if you don't have friends it can be hard to find fun things to do on the weekends, so definitely make an effort early on to meet interesting people so that you aren't stuck alone in your dorm on the weekends."
print(KNNPredict(text))

KNN Accuracy: 0.46153846153846156
not_low_ses


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from collections import Counter

# Check class distribution
print("Training labels distribution:", Counter(train_labels))

# Initialize and fit the Decision Tree model, with class weighting if imbalance exists
dt_model = DecisionTreeClassifier(random_state=42, max_depth=10, class_weight='balanced')
dt_model.fit(X_train, train_labels)

# Make predictions on the validation set
dt_pred = dt_model.predict(X_test)

# Calculate the performance metrics
dt_accuracy = accuracy_score(val_labels, dt_pred)
dt_f1 = f1_score(val_labels, dt_pred, average='binary')
dt_precision = precision_score(val_labels, dt_pred, average='binary')
dt_recall = recall_score(val_labels, dt_pred, average='binary')
dt_loss = 1 - dt_accuracy

print(f"Decision Tree Evaluation Loss: {dt_loss:.4f}")
print(f"Decision Tree Accuracy: {dt_accuracy:.4f}")

Training labels distribution: Counter({0: 51, 1: 51})
Decision Tree Evaluation Loss: 0.3846
Decision Tree Accuracy: 0.6154


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/SMU Work
import os
import nltk
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

# Load and preprocess the data
path = "GroundTruthFilter"
files = [file for file in os.listdir(path) if not file.startswith('.')]
ground_truth_texts = []
for file_name in files:
    with open(os.path.join(path, file_name), 'r') as f:
        output = f.read()
        ground_truth_texts.append(output)

nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

ground_truth_texts = [preprocess_text(item) for item in ground_truth_texts]

my_file = open("not-low-ses.txt", "r")
data = my_file.readlines()
my_file.close()

not_low_ses = []
for line in data:
    if len(line) > 100:
        not_low_ses.append(line)
    if len(not_low_ses) == len(ground_truth_texts):
        break

not_low_ses_texts = [preprocess_text(item) for item in not_low_ses]

# Combine the ground truth and not low SES texts into a single dataset
all_text = ground_truth_texts + not_low_ses_texts
all_label = [1 for i in range(len(ground_truth_texts))] + [-1 for i in range(len(not_low_ses_texts))]

# Split the dataset into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(all_text, all_label, test_size=0.2, random_state=42, shuffle=True)

# Vectorize the text data
vectorizer = CountVectorizer()
vectorizer.fit(train_texts)
X_train = vectorizer.transform(train_texts)
X_test = vectorizer.transform(val_texts)

# Initialize and fit the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, train_labels)

# Calculate the performance metrics
train_preds = rf_model.predict(X_train)
val_preds = rf_model.predict(X_test)

train_loss = 1 - accuracy_score(train_labels, train_preds)
val_loss = 1 - accuracy_score(val_labels, val_preds)

train_f1 = f1_score(train_labels, train_preds)
val_f1 = f1_score(val_labels, val_preds)

train_precision = precision_score(train_labels, train_preds)
val_precision = precision_score(val_labels, val_preds)

train_recall = recall_score(train_labels, train_preds)
val_recall = recall_score(val_labels, val_preds)

print(f"Evaluation Loss: {val_loss:.4f}")
print(f"F1 Score: {val_f1:.4f}")
print(f"Accuracy: {accuracy_score(val_labels, val_preds):.4f}")
print(f"Precision: {val_precision:.4f}")
print(f"Recall: {val_recall:.4f}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/SMU Work


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Evaluation Loss: 0.3462
F1 Score: 0.6897
Accuracy: 0.6538
Precision: 0.6250
Recall: 0.7692


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import time

# Ensure train-test split, vectorizer, X_train, X_test, train_labels, and val_labels are already defined.

# Helper function to evaluate models
def evaluate_model(model, X_train, X_test, train_labels, val_labels):
    # Train the model
    start_time = time.time()
    model.fit(X_train, train_labels)
    end_time = time.time()

    # Make predictions
    val_preds = model.predict(X_test)

    # Calculate metrics
    accuracy = accuracy_score(val_labels, val_preds)
    f1 = f1_score(val_labels, val_preds)
    precision = precision_score(val_labels, val_preds)
    recall = recall_score(val_labels, val_preds)
    runtime = end_time - start_time

    # Print metrics
    print(f"Model: {model.__class__.__name__}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"Training Runtime: {runtime:.4f} seconds")
    print("-" * 40)

# Initialize models
log_reg = LogisticRegression(random_state=42)
naive_bayes = MultinomialNB()
svm = SVC(kernel='linear', random_state=42)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)

# Evaluate Logistic Regression
evaluate_model(log_reg, X_train, X_test, train_labels, val_labels)

# Evaluate Naive Bayes
evaluate_model(naive_bayes, X_train, X_test, train_labels, val_labels)

# Evaluate SVM
evaluate_model(svm, X_train, X_test, train_labels, val_labels)

# Evaluate Random Forest
evaluate_model(rf_model, X_train, X_test, train_labels, val_labels)

# Evaluate Gradient Boosting
evaluate_model(gb_model, X_train, X_test, train_labels, val_labels)


Model: LogisticRegression
Accuracy: 0.8077
F1 Score: 0.7619
Precision: 1.0000
Recall: 0.6154
Training Runtime: 0.0257 seconds
----------------------------------------
Model: MultinomialNB
Accuracy: 0.8077
F1 Score: 0.8276
Precision: 0.7500
Recall: 0.9231
Training Runtime: 0.0043 seconds
----------------------------------------
Model: SVC
Accuracy: 0.8077
F1 Score: 0.7619
Precision: 1.0000
Recall: 0.6154
Training Runtime: 0.0123 seconds
----------------------------------------
Model: RandomForestClassifier
Accuracy: 0.6538
F1 Score: 0.6897
Precision: 0.6250
Recall: 0.7692
Training Runtime: 0.1720 seconds
----------------------------------------
Model: GradientBoostingClassifier
Accuracy: 0.6538
F1 Score: 0.6087
Precision: 0.7000
Recall: 0.5385
Training Runtime: 0.5599 seconds
----------------------------------------
